In [1]:
import numpy as np
from numpy import array as arr

from pap import summen_fehler
from pap import _rundung, _rundung_einzel, _negativ_wird_null, _nachkommastelle, _größenordnung, _erste_ziffer
from pap import _füllen, _entfernen, _zentrieren_rechts, _zahlen_ausrichtung, _plus_minus
from pap import _listen_transponieren

# pap.vergleichstabelle()-Entwicklung

In [3]:
'''
Benötigt:
* numpy als np
* np.array() als arr()

* pap.summen_fehler()
* pap._rundung()
* pap._negativ_wird_null()
* pap._nachkommastelle()
* pap._größenordnung()
* pap._listen_transponieren()
* pap._zahlenausrichtung()
* pap._füllen()
* pap._zentrieren_rechts()
* pap._plus_minus()
'''


def vergleichstabelle(werte, titel = '', einheit = '', größen = '', faktor = 1, ausrichtung = 'blöcke', 
                      beschreibung = 'standard'):
    '''
    Argumente
    ---------
    werte : np.ndarray (1D/2D, mit number_like oder None Elementen)
        Darf folgende Formen haben:
        * shape = (4, 0), np.array([ex_wert, ex_fehler, theo_wert, theo_fehler ])
        * shape = (4, N), np.array([ex_liste, ex_fehler_liste, theo_liste, theo_fehler_liste]) mit 
            Listenlänge N, N = Anzahl der Wertvergleiche, die gemacht werden sollen
        
        Art der Werte:
        Die Elemene dürfen reellwertige Zahlen sein (int, bool oder numpy-Äquivalente) oder None sein.
        Ein Fehler-Wert, der None ist, wird als 0 interpretiert. Ein ex-/theo-Wert der None ist, wird als 
        fehlende Zahl interpretiert und es wird kein Vergleich durchgeführt, siehe "Beispiele".
        
    titel : str, optional
    
    einheit : str, optional
    
    größen : str, array_like (1D, mit str Elementen), optional
        Überschriften für jeden Vergleich. Werden je nach Ausrichtung in der Titelzeile bzw. -spalte des 
        jeweiligen Vergleichs angezeigt.
        Darf folgende Formen haben:
        * str                   - einzelne Überschrift
        * Liste bis Länge N     - bis zu eine Überschrift für jeden Vergleich. Bei weniger Überschriften 
                                  bekommen die letzten Vergleiche keine Überschrift
        * Liste von Länge N + 1 - nur möglich, wenn einheit nicht angegeben. Erster String der Liste 
                                  ersetzt einheit und die Kennzeichnung 'Einheit' bzw. '[...]'. Die 
                                  restlichen Strings sind die jeweiligen Vergleichsüberschriften.
    
    faktor : number_like, optional
    
    ausrichtung : str, optional
        Darf nur sein:
        'blöcke'
        
        'liste'
    
    beschreibung : str, optional
        Darf nur sein:
        'standard'
        
        'knapp'
    '''
   


    # Überprüfen der Argumente
    werte_typ = type(werte)
    if werte_typ != np.ndarray:
        print(f'Eingabefehler: type(werte) = {werte_typ}')
        print('werte muss ein numpy Array sein.\n')
        return
    werte_shape = np.shape(werte)
    if (not len(werte_shape) in [1, 2]) or werte_shape[0] != 4:
            print(f'Eingabefehler: shape(werte) = {werte_shape}')
            print('werte muss die shape  (4, n) mit n = 0, 1, 2, ...  haben.\n')
            return
    
    titel_typ = type(titel) 
    if titel_typ != str:
        print(f'Eingabefehler: type(titel) = {titel_typ}')
        print('titel muss ein String sein.\n')
        return
    
    einheit_typ = type(einheit)
    if einheit_typ  != str:
        print(f'Eingabefehler: type(einheit) = {einheit_typ}')
        print('einheit muss ein String sein.\n')
        return
    
    # größen siehe: "# Vorbereitung Größenliste"
    
    ausrichtung_optionen = ['blöcke', 'liste']
    if not ausrichtung in ausrichtung_optionen:
        print(f'Eingabefehler: ausrichtung = {ausrichtung}')
        print(f'ausrichtung kann nur folgende Strings sein: {ausrichtung_optionen}\n')
        return
    
    beschreibung_optionen = ['standard', 'knapp']
    if not beschreibung in beschreibung_optionen:
        print(f'Eingabefehler: beschreibung = {beschreibung}')
        print(f'beschreibung kann nur folgende Strings sein: {beschreibung_optionen}\n')
        return
    
    
    
    # Vorbearbeitung der Argumente
    if np.shape(werte) == (4,):   
        werte = arr(werte, ndmin = 2).T
    
    ## Auslese von None-Elementen im Werte-Array
    test_kein_wert = werte == None
    none_stellen = np.where(test_kein_wert)
    nullen = np.zeros(np.shape(werte))
    werte = arr(np.where(test_kein_wert, nullen, werte), dtype = np.float64)
    ex_none = []
    theo_none = []
    for i in range(len(none_stellen[0])):
        if none_stellen[0][i] == 0:
            ex_none.append(none_stellen[1][i])
        if none_stellen[0][i] == 2:
            theo_none.append(none_stellen[1][i])
    alle_none = ex_none + theo_none
    
    ## Vorbereitung der zu verrechnenden Werte
    werte = werte * faktor   # Umrechnung auf gewünschte Größenordnung bzw. Einheit
    werte[1::2] = np.abs(werte[1::2])      # Keine negativen Fehlerwerte
    ex, ex_fehler, theo, theo_fehler = werte
    
    ## Wichtige Konstanten
    anzahl_vergleiche = np.shape(werte)[1]
    anzahl_zahlen = 8    # Anzahl Zahlen, die in einem Vergleich vorkommen
    anzahl_vergleichswerte = 5  # Anzahl (fehlerbehafteter) Werte, die in einem Vergleich vorkommen
    
    ## Vorberarbeitung der Größen(-liste)
    größe0 = None
    if not type(größen) in [str, list]:
        größen = list(größen)
    if len(größen) == 0:
        if einheit == '':
            überschriften = []
        else:
            fehlende_größen = [''] * anzahl_vergleiche
            überschriften = [*fehlende_größen]
    elif type(größen) == str:
        fehlende_größen = [''] * (anzahl_vergleiche - 1)
        überschriften = [größen, *fehlende_größen]
    elif len(größen) < anzahl_vergleiche:
        fehlende_größen = [''] * (anzahl_vergleiche - len(größen))
        überschriften = [*größen, *fehlende_größen]
    elif len(größen) == anzahl_vergleiche:
        überschriften = größen
    elif len(größen) - 1 == anzahl_vergleiche and einheit == '':
        größe0 = größen.pop(0)
        überschriften = größen
    else:
        if anzahl_vergleiche == 1:
            print(f'Eingabefehler: len(größen) = {len(größen)}, aber es findet nur {anzahl_vergleiche} Vergleich statt.')
        else:
            print(f'Eingabefehler: len(größen) = {len(größen)}, aber es finden nur {anzahl_vergleiche} Vergleiche statt.')
        print('Es darf höchstens so viele Größen wie Vergleiche geben.')
        print('(Ausnahme: einheit = \'\' und len(größen) = Anzahl Vergleiche + 1,')
        print(' dann wird der erste Wert als einheit interpretiert)\n')
        return
        
    
    # Berechnung und Rundung
    ## Ausrechnen der Abweichungen
    abweichung_abs = ex - theo
    abweichung_abs_fehler = summen_fehler(arr([theo_fehler, ex_fehler]))
    nullen = np.zeros((anzahl_vergleiche, ))
    abweichung_rel = arr([abweichung_abs[i] / theo[i] * 100  if theo[i] != 0  else 0 
                          for i in range(anzahl_vergleiche)])   # [%]
    abweichung_sig = arr([np.abs(abweichung_abs[i] / abweichung_abs_fehler[i])  
                          if abweichung_abs_fehler[i] != 0  else 0  for i in range(anzahl_vergleiche)])
    
    
    ## Bestimmung der Nachkommastellen
    werte_absolut = arr([ex, theo, abweichung_abs])
    werte_fehler  = arr([ex_fehler, theo_fehler, abweichung_abs_fehler])
    werte_relativ = arr([abweichung_rel, abweichung_sig])
    
    nachkommastellen_fehler = _nachkommastelle(werte_fehler, sig_grenze = 3.95)
    nachkommastellen_absolut = nachkommastellen_fehler
    nachkommastellen_relativ = _nachkommastelle(werte_relativ, sig_stellen = 2, sig_grenze = 1.0)
    
    nachkommastellen = []
    for i in range(3):
        nachkommastellen.append(nachkommastellen_absolut[i])
        nachkommastellen.append(nachkommastellen_fehler[i])
    nachkommastellen.extend(nachkommastellen_relativ)
    nachkommastellen = np.array(nachkommastellen, dtype = int)
    
    
    ## Rundung aller Werte
    alle_werte = arr([ex, ex_fehler, theo, theo_fehler, abweichung_abs, abweichung_abs_fehler,
                      abweichung_rel, abweichung_sig])
    alle_werte_gerundet = _rundung(alle_werte, nachkommastellen)
    größenordnungen     = _größenordnung(alle_werte_gerundet)
    
    
    ## Darstellung der einzelnen Werte
    alle_werte_strings = []
    for i in range(anzahl_zahlen):
        zahlen_strings = []
        for j in range(anzahl_vergleiche):
            if np.abs(größenordnungen[i][j]) < 5:
                nachkommastelle = _negativ_wird_null(nachkommastellen[i][j])
                wert_string = '{:.{prec}f}'.format(alle_werte_gerundet[i][j], prec = nachkommastelle)
            else:
                größenordnung = größenordnungen[i][j]
                wert_gestutzt = alle_werte_gerundet[i][j] * 10**(-float(größenordnung))
                präzision = nachkommastellen[i][j] + größenordnung

                wert_string = '{:.{prec}f}e{}'.format(wert_gestutzt, größenordnung, prec = präzision)
            zahlen_strings.append(wert_string)
        alle_werte_strings.append(zahlen_strings)
    
    alle_werte_strings = arr(alle_werte_strings).T
    for i in range(anzahl_vergleiche):
        if i in ex_none:
            alle_werte_strings[i][0:2] = ['', '']
        if i in theo_none:
            alle_werte_strings[i][2:4] = ['', '']
        if i in alle_none:
            alle_werte_strings[i][4:8] = ['', '', '', '']

    
    
    # Formatierung der Werte zu schönen, gut lesbaren Vergleichen
    if ausrichtung == 'blöcke':
        alle_vergleiche = []
        for i in range(anzahl_vergleiche):
            strings_einzelwerte = alle_werte_strings[i]

            # Anpassen der Absolutwerte aneinander
            strings_absolut = list(strings_einzelwerte[[0, 2, 4]])
            strings_absolut = _zahlen_ausrichtung(strings_absolut)

            # Anpassen der Fehlerwerte aneinander
            strings_fehler = list(strings_einzelwerte[[1, 3, 5]])
            strings_fehler = _zahlen_ausrichtung(strings_fehler)

            # Zusammenstellen des Vergleichs
            plus_minus = _plus_minus(strings_absolut)
            strings_plusminus = [strings_absolut[j] + plus_minus[j] + strings_fehler[j]
                                 for j in range(len(strings_absolut))]
            
            string_rel, string_sig = [strings_einzelwerte[6] + ' %', strings_einzelwerte[7] + ' σ']
            if string_rel[0] == '-':
                string_sig = _füllen(string_sig, index = 0)
            if theo[i] == 0:
                string_rel = '-'
            if abweichung_abs_fehler[i] == 0:
                string_sig = '-'
            if i in alle_none:
                string_rel = ''
                string_sig = ''
                
            strings_vergleich = [*strings_plusminus, string_rel, string_sig]
            breite_block = max([len(string) for string in strings_vergleich])
            strings_vergleich = [_zentrieren_rechts(string, breite_block) for string in strings_vergleich]
            alle_vergleiche.append(strings_vergleich)
            
    elif ausrichtung == 'liste':
        alle_werte_strings = alle_werte_strings.T
            
        zahlen_absolut = [_zahlen_ausrichtung(list(zahlen)) for zahlen in alle_werte_strings[:6]]
        plus_minus = [_plus_minus(zahlen_absolut[i]) for i in range(0, len(zahlen_absolut), 2)]
        strings_absolut = [[zahlen_absolut[i][j] + plus_minus[i//2][j] + zahlen_absolut[i + 1][j] 
                            for j in range(anzahl_vergleiche)] for i in range(0, len(zahlen_absolut), 2)]
        
        strings_rel = [alle_werte_strings[6][i]  + ' %'  if theo[i] != 0  else '-'  
                       for i in range(anzahl_vergleiche)]
        strings_sig = [alle_werte_strings[7][i] + ' σ'  if abweichung_abs_fehler[i] != 0  else '-'  
                       for i in range(anzahl_vergleiche)]
        strings_rel = ['' if i in alle_none else strings_rel[i] for i in range(anzahl_vergleiche)]
        strings_sig = ['' if i in alle_none else strings_sig[i] for i in range(anzahl_vergleiche)]
        strings_rel = _zahlen_ausrichtung(strings_rel)
        strings_sig = _zahlen_ausrichtung(strings_sig)
        
        alle_vergleiche = [*strings_absolut, strings_rel, strings_sig]
        
    
    
    # Erstellen der fertigen Tabellenstrings
    if titel != '':
        titel_strings = [titel, '']
    else:
        titel_strings = []
    
    
    if ausrichtung == 'blöcke':
        separator = '   '
        
        # Angleichen der Länge der Größenstrings mit der Breite der dazugehörigen Vergleichsblöcke
        if überschriften != []:
            
            breite_blöcke = [max(len(alle_vergleiche[i][0]), len(überschriften[i])) 
                             for i in range(anzahl_vergleiche)]
            überschriften = ['{:^{width}}'.format(überschriften[i], width = breite_blöcke[i]) 
                             for i in range(anzahl_vergleiche)]
            alle_vergleiche = [['{:^{width}}'.format(vergleichswert, width = breite_blöcke[i]) 
                                for vergleichswert in alle_vergleiche[i]] for i in range(anzahl_vergleiche)]
            
        # Erstellen vom "Einheiten und Beschreibungen"-Block (Überschriftenspalte)
        if beschreibung == 'standard':
            beschreibung_liste = ['Wert       experimentell', '           theoretisch  ', 'Abweichung absolut      ', 
                                  '           relativ      ', '           sigma        ']
            if einheit != '':
                einheit_string = f'Einheit    {einheit}'
                überschuss     = len(einheit) - len('experimentell')
                stelle         = 'rechts'
                
        elif beschreibung == 'knapp':
            beschreibung_liste = ['Wert exp', '     the', 'Abw. abs', '     rel', '     sig']
            if einheit != '':
                einheit_string = f'[{einheit}]'
                überschuss     = len(einheit_string) - len(beschreibung_liste[0])
                stelle         = 'links'
            
        if größe0 != None:
            einheit_string = größe0
            überschuss     = len(einheit_string) - len(beschreibung_liste[0])
            stelle         = 'rechts'
            
        if einheit == '' and größe0 == None:
            einheit_string = ' ' * len(beschreibung_liste[0])
        elif überschuss < 0:
            einheit_string = _füllen(einheit_string, stelle, menge = -überschuss)
        elif überschuss > 0:
            beschreibung_liste = [_füllen(string, index = stelle, menge = überschuss) for string in beschreibung_liste]
        
        
        # Erstellen und Sammeln aller Überschriften-Strings
        if überschriften != []:
            überschriften.insert(0, einheit_string)
            überschriften_string = separator.join(überschriften)   
            titel_strings.append(überschriften_string)
            
        ## Erstellen der Linie, die Überschriften und Vergleiche trennt
        alle_vergleiche   = _listen_transponieren(alle_vergleiche)
        blöcke_längen = [len(vergleich) for vergleich in alle_vergleiche[0]]
        blöcke_längen.insert(0, len(einheit_string))
        oberlinie = separator.join(['-' * länge for länge in blöcke_längen])
        titel_strings.append(oberlinie)
        
        # Erstellen aller Vergleich-Strings
        vergleich_strings = []
        for i in range(anzahl_vergleichswerte):
            vergleich_string = separator.join([beschreibung_liste[i], *alle_vergleiche[i]])
            vergleich_strings.append(vergleich_string)
    
    
    if ausrichtung == 'liste':
        separator = ' | '
        
        if beschreibung == 'standard':
            beschreibung_liste = [['Wert', 'Abweichung'], 
                                  ['experimentell', 'theoretisch', 'absolut', 'relativ', 'sigma']]
            
        elif beschreibung == 'knapp':
            beschreibung_liste = [['Wert', 'Abw.'], ['exp', 'the', 'abs', 'rel', 'sig']]
            if einheit != '':
                einheit = f'[{einheit}]'
        
        for i in range(anzahl_vergleichswerte):
            differenz = len(beschreibung_liste[1][i]) - len(alle_vergleiche[i][0])
            if differenz < 0:
                beschreibung_liste[1][i] = '{:^{width}}'.format(beschreibung_liste[1][i], width = len(alle_vergleiche[i][0]))
            elif differenz > 0:
                alle_vergleiche[i] = [_füllen(string, 'links', menge = differenz) for string in alle_vergleiche[i]]
        
        beschreibung_liste[0][0] = '{:^{width}}'.format(beschreibung_liste[0][0], 
                                                        width = len(separator.join(beschreibung_liste[1][:2])))
        beschreibung_liste[0][1] = '{:^{width}}'.format(beschreibung_liste[0][1], 
                                                        width = len(separator.join(beschreibung_liste[1][2:])))
        
        if überschriften != []:  # Wenn falsch, bedeutet auch, dass einheit == '' ist.
            if einheit == '' or beschreibung == 'knapp':
                überschriften.insert(0, '')
            elif beschreibung == 'standard':
                überschriften.insert(0, 'Einheit')
            if größe0 == None:
                überschriften.insert(1, einheit)
            else:
                überschriften.insert(1, größe0)
        
            überschrift_längen = [len(string) for string in überschriften]
            überschriften_breite = max(überschrift_längen)
            for i in range(2):
                einheit_string = '{:^{width}}'.format(überschriften.pop(0), width = überschriften_breite)
                beschreibung_liste[i].insert(0, einheit_string)
            überschriften = ['{:<{width}}'.format(string, width = überschriften_breite) for string in überschriften]
            
        
        # Erstellen und Sammeln aller Überschriften-Strings
        for i in range(2):
            beschreibung_string = separator.join(beschreibung_liste[i])
            titel_strings.append(beschreibung_string)
        oberlinie = '-' * len(beschreibung_string)
        titel_strings.append(oberlinie)
        
        # Erstellen aller Vergleich-Strings
        alle_vergleiche   = _listen_transponieren(alle_vergleiche)
        if überschriften != []:
            vergleich_liste = [[überschriften[i], *alle_vergleiche[i]] for i in range(anzahl_vergleiche)]
        else:
            vergleich_liste = alle_vergleiche
        vergleich_strings = [separator.join(vergleich) for vergleich in vergleich_liste]
    
    
    # Printen der Tabelle
    alle_strings = [*titel_strings, *vergleich_strings]
    for zeile in alle_strings:
        print(zeile)  
    print('')

 ## Tests

### Eingabefehler

In [4]:
# Test: Eingabefehler aller Art

# werte ist nicht np.array
vergleichstabelle([1, 2, 3, 4])


# werte hat zu viele Dimensionen
vergleichstabelle(np.ones((4, 2, 0)))

# werte hat falsche shape
vergleichstabelle(np.ones((5, 3)))


werte = arr([12, 4, 6, 4])
# titel ist nicht str
vergleichstabelle(werte, titel = 1)

# einheit ist nicht str
vergleichstabelle(werte, einheit = arr(1.))

# größenliste ist zu lang
vergleichstabelle(werte, größen = ['a', 'b', 'c'])  # Test singular
vergleichstabelle(werte[:, np.newaxis] * np.ones((4, 2)), größen = ['a', 'b', 'c', 'd']) # Test plural

# ausrichtung ist keine der möglichen Optionen
vergleichstabelle(werte, ausrichtung = 2.)

# beschreibung ist keine der möglichen Optionen
vergleichstabelle(werte, beschreibung = range(1))

Eingabefehler: type(werte) = <class 'list'>
werte muss ein numpy Array sein.

Eingabefehler: shape(werte) = (4, 2, 0)
werte muss die shape  (4, n) mit n = 0, 1, 2, ...  haben.

Eingabefehler: shape(werte) = (5, 3)
werte muss die shape  (4, n) mit n = 0, 1, 2, ...  haben.

Eingabefehler: type(titel) = <class 'int'>
titel muss ein String sein.

Eingabefehler: type(einheit) = <class 'numpy.ndarray'>
einheit muss ein String sein.

Eingabefehler: len(größen) = 3, aber es findet nur 1 Vergleich statt.
Es darf höchstens so viele Größen wie Vergleiche geben.
(Ausnahme: einheit = '' und len(größen) = Anzahl Vergleiche + 1,
 dann wird der erste Wert als einheit interpretiert)

Eingabefehler: len(größen) = 4, aber es finden nur 2 Vergleiche statt.
Es darf höchstens so viele Größen wie Vergleiche geben.
(Ausnahme: einheit = '' und len(größen) = Anzahl Vergleiche + 1,
 dann wird der erste Wert als einheit interpretiert)

Eingabefehler: ausrichtung = 2.0
ausrichtung kann nur folgende Strings sein: [

In [5]:
# Test ob größen Einzelstrings als str, list, tupel und np.array aufnehmen kann
inhalt = [ '', 'Hi!']
for element in inhalt:
    array_liste = [element, [element], (element), arr([element])]
    for array in array_liste:
        vergleichstabelle(werte, größen = array)


------------------------   ------
Wert       experimentell   12 ± 4
           theoretisch      6 ± 4
Abweichung absolut          6 ± 6
           relativ          100 %
           sigma            1.1 σ

                                 
------------------------   ------
Wert       experimentell   12 ± 4
           theoretisch      6 ± 4
Abweichung absolut          6 ± 6
           relativ          100 %
           sigma            1.1 σ

------------------------   ------
Wert       experimentell   12 ± 4
           theoretisch      6 ± 4
Abweichung absolut          6 ± 6
           relativ          100 %
           sigma            1.1 σ

                                 
------------------------   ------
Wert       experimentell   12 ± 4
           theoretisch      6 ± 4
Abweichung absolut          6 ± 6
           relativ          100 %
           sigma            1.1 σ

                            Hi!  
------------------------   ------
Wert       experimentell   12 ± 4
          

In [6]:
# Test ob größen eine String-Liste als list, tupel und np.array aufnehmen kann

werte = arr([[374, 23, 365, 0.4], [287, 17, 302, 0.5]]).T
inhalt = [['a', 'b'], ['Hulp', 'a', 'b']]
for element in inhalt:
    array_liste = [element, tuple(element), arr(element)]
    for array in array_liste:
        vergleichstabelle(werte, größen = array)



                                a              b      
------------------------   ------------   ------------
Wert       experimentell   374   ± 23     287   ± 17  
           theoretisch     365.0 ±  0.4   302.0 ±  0.5
Abweichung absolut           9   ± 23     -15   ± 17  
           relativ             2.5 %         -5.0 %   
           sigma              0.39 σ          0.88 σ  

                                a              b      
------------------------   ------------   ------------
Wert       experimentell   374   ± 23     287   ± 17  
           theoretisch     365.0 ±  0.4   302.0 ±  0.5
Abweichung absolut           9   ± 23     -15   ± 17  
           relativ             2.5 %         -5.0 %   
           sigma              0.39 σ          0.88 σ  

                                a              b      
------------------------   ------------   ------------
Wert       experimentell   374   ± 23     287   ± 17  
           theoretisch     365.0 ±  0.4   302.0 ±  0.5
Abweichu

### Berechnung, Rundung und Wertausrichtung

In [7]:
# Test: Darstellung von Werten verschiedenster Größenordnungen

zahlliste = arr([1e100, 3849184e38, -1.3984820e5, 48024.304, -40000, -39999.0, 1023, 483, -395.00, 23.30489, 
                 4.09, 3.688, 1.3848, -0.839, 0.094, 0.0003848, -0.00004848, 5.4839e-5, -3.9485e-7, 7.38e-99])
fehlerliste = zahlliste / 77
theoliste = zahlliste * 1.05

alle_werte = arr([zahlliste, fehlerliste, theoliste, fehlerliste]).T
for i in range(len(zahlliste)):
    vergleichstabelle(alle_werte[i], beschreibung = 'knapp')

--------   -------------------
Wert exp    1.000e100 ± 1.3e98
     the    1.050e100 ± 1.3e98
Abw. abs   -5.0  e 98 ± 1.8e98
     rel          -4.8 %      
     sig           2.7 σ      

--------   ---------------
Wert exp    3.85e44 ± 5e42
     the    4.04e44 ± 5e42
Abw. abs   -1.9 e43 ± 7e42
     rel        -4.8 %    
     sig         2.7 σ    

--------   -----------------
Wert exp     -1.398e5 ± 1800
     the     -1.468e5 ± 1800
Abw. abs   7000       ± 2600
     rel         -4.8 %     
     sig          2.7 σ     

--------   -----------
Wert exp   48000 ± 600
     the   50400 ± 600
Abw. abs   -2400 ± 900
     rel      -4.8 %  
     sig       2.7 σ  

--------   ------------
Wert exp   -40000 ± 500
     the   -42000 ± 500
Abw. abs     2000 ± 700
     rel      -4.8 %   
     sig       2.7 σ   

--------   ------------
Wert exp   -40000 ± 500
     the   -42000 ± 500
Abw. abs     2000 ± 700
     rel      -4.8 %   
     sig       2.7 σ   

--------   ---------
Wert exp   1023 ± 13
    

In [8]:
# Test: Modifikation um Darstellung von negativen Zahlen weiter zu testen
theoliste[0] = theoliste[0] * 0.9    # Test "kein -"
theoliste[-4] = theoliste[-4] * 0.9  # Test "alle -"

alle_werte = arr([zahlliste, fehlerliste, theoliste, fehlerliste]).T
for i in [0, -4]:
    vergleichstabelle(alle_werte[i], beschreibung = 'knapp')

--------   ------------------
Wert exp   1.000e100 ± 1.3e98
     the   9.45 e 99 ± 1.3e98
Abw. abs   5.5  e 98 ± 1.8e98
     rel          5.8 %      
     sig          3.0 σ      

--------   ---------------
Wert exp   -4.85e-5 ± 6e-7
     the   -4.58e-5 ± 6e-7
Abw. abs   -2.7 e-6 ± 9e-7
     rel        5.8 %     
     sig        3.0 σ     



In [9]:
# Test mit normalen Werten
vergleichstabelle(arr([12.3, 0.12, 10.98, 0.987]), beschreibung = 'knapp')

--------   ------------
Wert exp   12.30 ± 0.12
     the   11.0  ± 1.0 
Abw. abs    1.3  ± 1.0 
     rel       12 %    
     sig       1.3 σ   



In [10]:
# Tests mit negativen Werten und negativen Fehlern
vergleichstabelle(arr([ 0.123, -0.012,  1.098,  0.0987]), beschreibung = 'knapp')
vergleichstabelle(arr([-0.123,  0.012, -1.098,  0.0987]), beschreibung = 'knapp')
vergleichstabelle(arr([ 0.123,  0.012, -1.098, -0.0987]), beschreibung = 'knapp')

--------   --------------
Wert exp    0.123 ± 0.012
     the    1.10  ± 0.10 
Abw. abs   -0.98  ± 0.10 
     rel        -89 %    
     sig        9.8 σ    

--------   --------------
Wert exp   -0.123 ± 0.012
     the   -1.10  ± 0.10 
Abw. abs    0.98  ± 0.10 
     rel        -89 %    
     sig        9.8 σ    

--------   --------------
Wert exp    0.123 ± 0.012
     the   -1.10  ± 0.10 
Abw. abs    1.22  ± 0.10 
     rel       -110 %    
     sig         12 σ    



In [11]:
# Tests mit der Positionierung von Minuszeichen
vergleichstabelle(arr([1248.834, 48.489, -93.17, 25.37]), beschreibung = 'knapp')

--------   ---------
Wert exp   1250 ± 50
     the    -93 ± 25
Abw. abs   1340 ± 50
     rel    -1400 % 
     sig      25 σ  



In [12]:
# Test mit längeren relativen als ±-Strings und dortigen Minuspositionen
vergleichstabelle(arr([1e-7, 4,  1e-5, 4]), beschreibung = 'knapp')
vergleichstabelle(arr([5.7321, 4 ,  -1e-2, 4]), beschreibung = 'knapp')

--------   ---------
Wert exp      0 ± 4 
     the      0 ± 4 
Abw. abs     -0 ± 6 
     rel     -99 %  
     sig    1.8e-6 σ

--------   --------
Wert exp     6 ± 4 
     the    -0 ± 4 
Abw. abs     6 ± 6 
     rel   -57000 %
     sig     1.0 σ 



In [13]:
# Test mit der Minus-Position bei negativen Exponenten 
# (siehe auch großer Listendarstellung-Test, Vergleich ganz unten)
## Grenzen der Rechenpräzision hier sichtbar

vergleichstabelle(arr([1.000000000000002e10, 7e-10, 1.000000000000001e10, 7e-10]))

------------------------   -----------------------------------
Wert       experimentell   1.00000000000000199840e10 ± 7  e-10
           theoretisch     1.00000000000000088818e10 ± 7  e-10
Abweichung absolut         9.5367                e-6 ± 1.0e -9
           relativ                      9.5e-14 %             
           sigma                          9600 σ              



#### Rundung von 0-Werten

In [14]:
werte = arr([[0, 1.0e-3, 218.57, 6.4], [382.383, 0.00e8, 218.57, 6.4], 
                       [382.383, 6.4e-4, 218.573947, 0], [0, 0.0, 0, 0.00e-3]]).T
größen = ['Ex-Wert = 0', 'Ex-Fehler = 0', 'Theo-Fehler = 0', 'Alles 0']
vergleichstabelle(werte, titel = 'Rundungstests',
                  größen = größen, beschreibung = 'knapp')
vergleichstabelle(werte, titel = 'Rundungstests',
                  größen = größen, beschreibung = 'standard', ausrichtung = 'liste')

Rundungstests

              Ex-Wert = 0       Ex-Fehler = 0    Theo-Fehler = 0     Alles 0 
--------   ------------------   -------------   -----------------   ---------
Wert exp      0.0000 ± 0.0010    382.4 ± 0.0    382.3830 ± 0.0006   0.0 ± 0.0
     the    219      ± 6         219   ± 6      218.6    ± 0.0      0.0 ± 0.0
Abw. abs   -219      ± 6         164   ± 6      163.8091 ± 0.0006   0.0 ± 0.0
     rel         -100 %              75 %              75 %             -    
     sig           34 σ              26 σ            2.6e5 σ            -    

Rundungstests

                |              Wert               |               Abweichung               
                |   experimentell   | theoretisch |      absolut       | relativ |  sigma  
-------------------------------------------------------------------------------------------
Ex-Wert = 0     |   0.0000 ± 0.0010 | 219   ± 6   | -219      ± 6      |  -100 % | 34     σ
Ex-Fehler = 0   | 382.4    ± 0.0    | 219   ± 6   |  16

#### Berechnung (Ausnahmefall)

In [15]:
testwerte = arr([[382.383, 3.2, 0, 6.4], [382.383, 0, 218.57, 0], [0, 0.0, 0, 0.00e-3]]).T
größen = ['Theo-Wert = 0', 'Abw-Fehler = 0', 'Alles = 0']
vergleichstabelle(testwerte, titel = 'Crashtests', größen = größen, beschreibung = 'knapp')
print('')
vergleichstabelle(testwerte, titel = 'Crashtests', größen = größen, beschreibung = 'knapp', 
                  ausrichtung = 'liste')

Crashtests

           Theo-Wert = 0   Abw-Fehler = 0   Alles = 0
--------   -------------   --------------   ---------
Wert exp    382.4 ± 3.2     382.4 ± 0.0     0.0 ± 0.0
     the      0   ± 6       218.6 ± 0.0     0.0 ± 0.0
Abw. abs    382   ± 7       163.8 ± 0.0     0.0 ± 0.0
     rel         -              75 %            -    
     sig        53 σ             -              -    


Crashtests

               |           Wert            |           Abw.           
               |     exp     |     the     |     abs     | rel  | sig 
----------------------------------------------------------------------
Theo-Wert = 0  | 382.4 ± 3.2 |   0   ± 6   | 382   ± 7   |   -  | 53 σ
Abw-Fehler = 0 | 382.4 ± 0.0 | 218.6 ± 0.0 | 163.8 ± 0.0 | 75 % |   - 
Alles = 0      |   0.0 ± 0.0 |   0.0 ± 0.0 |   0.0 ± 0.0 |   -  |   - 



#### Präzision exakter Theowerte

In [16]:
vergleichstabelle(arr([382.383, 6.4e-4, 218.573947, 0]), titel = 'Theo-Fehler = 0, Rundungstest', beschreibung = 'knapp')

'''
Stand 2020-10-04: Exakter Theo-Wert nicht manuell rundbar
Präzision eines exakten Theo-Wertes ist auf immer eine Nachkommastelle beschränkt.

Dieser Wert sollte in zukunft manuell gerundet werden können.

the   218.6    ± 0.0 
''';

Theo-Fehler = 0, Rundungstest

--------   -----------------
Wert exp   382.3830 ± 0.0006
     the   218.6    ± 0.0   
Abw. abs   163.8091 ± 0.0006
     rel          75 %      
     sig        2.6e5 σ     



#### Blockdarstellung

In [17]:
# Test mit zwei Vergleichen
vergleichstabelle(arr([[0.123, 400 / 7], 
                       [0.012, 23 / 7], 
                       [1.098, 337 / 7], 
                       [0.987, 29 / 7]]), beschreibung = 'knapp')

--------   --------------   ----------
Wert exp    0.123 ± 0.012   57.1 ± 3.3
     the    1.1   ± 1.0     48   ± 4  
Abw. abs   -1.0   ± 1.0      9   ± 5  
     rel        -89 %          19 %   
     sig        0.99 σ         1.7 σ  



In [18]:
# Test der Rundung (auch bei rel. und sig. Abweichung)
vergleichstabelle(arr([[284.3858, 0.07538, 0.00829339400, -0.123], 
                       [40.0000 , 0.00400, 0.00003950000,  0.0012], 
                       [317.3048, 0.07536, 0.00999999900, -1.098], 
                       [39.9999 , 0.00399, 0.00003949990,  0.0000098]]), beschreibung = 'knapp')

--------   --------   -------------   ------------------   ---------------------
Wert exp   280 ± 40   0.075 ± 0.004    0.008293 ± 4.0e-5   -0.1230   ± 0.0012   
     the   320 ± 40   0.075 ± 0.004    0.010000 ± 3.9e-5   -1.098000 ± 1.0   e-5
Abw. abs   -30 ± 60   0.000 ± 0.006   -0.00171  ± 6  e-5    0.9750   ± 0.0012   
     rel     -10 %       0.027 %             -17 %                 -89 %        
     sig     0.58 σ      0.0035 σ             31 σ                  810 σ       



#### Listendarstellung

In [19]:
# Test: Listendarstellung der Vergleiche mit vielen Größenordnungen

# Um den Output breit genug für die Tabelle zu machen.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [20]:
# Großer Test der Ausrichtungsfeatures

vergleichstabelle(alle_werte.T, ausrichtung = 'liste')

                                 Wert                                   |                     Abweichung                    
          experimentell            |            theoretisch             |             absolut              | relativ | sigma
----------------------------------------------------------------------------------------------------------------------------
     1.000   e100 ±    1.3   e  98 |      9.45    e 99 ±    1.3   e  98 |     5.5   e  98 ±    1.8   e  98 |   5.8 % | 3.0 σ
     3.85    e 44 ±    5     e  42 |      4.04    e 44 ±    5     e  42 |    -1.9   e  43 ±    7     e  42 |  -4.8 % | 2.7 σ
    -1.398   e  5 ± 1800           |     -1.468   e  5 ± 1800           |  7000           ± 2600           |  -4.8 % | 2.7 σ
 48000            ±  600           |  50400            ±  600           | -2400           ±  900           |  -4.8 % | 2.7 σ
-40000            ±  500           | -42000            ±  500           |  2000           ±  700           |  -4.8 % | 2.7 σ


In [21]:
# Test: Listendarstellungen bei kleineren Listen und ähnlicheren Werten.

vergleichstabelle(alle_werte[7:10].T, ausrichtung = 'liste')
vergleichstabelle(alle_werte[0:2].T, ausrichtung = 'liste')
vergleichstabelle(alle_werte[2:5].T, ausrichtung = 'liste')

             Wert               |          Abweichung          
experimentell  |  theoretisch   |   absolut   | relativ | sigma
---------------------------------------------------------------
 483    ± 6    |  507    ± 6    | -24   ± 9   |  -4.8 % | 2.7 σ
-395    ± 5    | -415    ± 5    |  20   ± 7   |  -4.8 % | 2.7 σ
  23.30 ± 0.30 |   24.47 ± 0.30 |  -1.2 ± 0.4 |  -4.8 % | 2.7 σ

                Wert                  |             Abweichung            
  experimentell    |   theoretisch    |     absolut      | relativ | sigma
--------------------------------------------------------------------------
1.000e100 ± 1.3e98 | 9.45e99 ± 1.3e98 |  5.5e98 ± 1.8e98 |   5.8 % | 3.0 σ
3.85 e 44 ± 5  e42 | 4.04e44 ± 5  e42 | -1.9e43 ± 7  e42 |  -4.8 % | 2.7 σ

                  Wert                    |           Abweichung          
   experimentell    |     theoretisch     |   absolut    | relativ | sigma
--------------------------------------------------------------------------
    -1.398e5 ±

In [22]:
# Test: Rundung am Cutoff
# Test: Vergleich der Block-Darstellung mit der Listendarstellung

zahlliste = arr([39950, -0.000394999, -0.000395000, 0.000395001, 0.0000395000])
fehlerliste = zahlliste / 10
theoliste = zahlliste * 1.01
alle_werte = arr([zahlliste, fehlerliste, theoliste, fehlerliste * 1.01])

vergleichstabelle(alle_werte, ausrichtung = 'blöcke')
vergleichstabelle(alle_werte, ausrichtung = 'liste')

'''
Stand 2020-10-04: Fehler bei den experimentellen Werten
Rundung bei Vergleichen 1, 2 und 4 läuft wie erwartet.
Vergleich 3 enthält sowohl einen Rundungsfehler (float-Problem) als auch einen Cutoff-Fehler (Grund bisher unbekannt).
Vergleich 5 enthält wieder den Rundungsfehler aber der Cutoff wurde hier richtig fortgeführt.

Da dies Extremfälle sind (tritt nur bei Fehlerzahlen, die mit 3.950 (Cutoff) beginnen auf), werden sie fürs erste ignoriert.

       experimentell        
----------------------------
40000           ± 4000      
   -0.000395    ±    3.9e-5 
   -0.000395    ±    4.0e-5 
    0.00040     ±    4  e-5 
    3.95    e-5 ±    3.9e-6 

''';

------------------------   ------------   ------------------   ------------------   ---------------   ---------------------
Wert       experimentell   40000 ± 4000   -0.000395 ± 3.9e-5   -0.000395 ± 4.0e-5    0.00040 ± 4e-5    3.95    e-5 ± 3.9e-6
           theoretisch     40000 ± 4000   -0.00040  ± 4  e-5   -0.00040  ± 4  e-5    0.00040 ± 4e-5    4.0     e-5 ± 4  e-6
Abweichung absolut            -0 ± 6000    0.00000  ± 6  e-5    0.00000  ± 6  e-5   -0.00000 ± 6e-5   -0.000000    ± 6  e-6
           relativ            -0.99 %           -0.99 %              -0.99 %            -0.99 %              -0.99 %       
           sigma              0.070 σ           0.070 σ              0.070 σ             0.070 σ              0.070 σ      

                         Wert                          |               Abweichung               
       experimentell        |       theoretisch        |       absolut       | relativ |  sigma 
-------------------------------------------------------------

### Tests der Beschriftung und Darstellung mit richtigen Werten aus PAP-Experimenten

In [23]:
# Test: standard Wertdarstellung mit Einheit und Titel
werte = arr([75.31709457, 0.76723428, 66., 0.5])
vergleichstabelle(werte, titel = 'Literaturvergleich Inklinationswinkel', einheit = '°')
vergleichstabelle(werte, titel = 'Literaturvergleich', einheit = '°', größen = 'Inklinationswinkel', beschreibung = 'knapp')


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 5: Literaturvergleich Inklinationswinkel

Angaben in °
-------------------  ----
Messwert             75.3
dessen Fehler         0.8
Literaturwert        66.0
dessen Fehler         0.5
rel. Abweichung [%]  14.1
Sigma                10.2
''';

Literaturvergleich Inklinationswinkel

Einheit    °                         
------------------------   ----------
Wert       experimentell   75.3 ± 0.8
           theoretisch     66.0 ± 0.5
Abweichung absolut          9.3 ± 0.9
           relativ            14 %   
           sigma              10 σ   

Literaturvergleich

     [°]   Inklinationswinkel
--------   ------------------
Wert exp       75.3 ± 0.8    
     the       66.0 ± 0.5    
Abw. abs        9.3 ± 0.9    
     rel          14 %       
     sig          10 σ       



In [24]:
# Test: standard Wertdarstellung 
## mit unterschiedlich vielen Nachkommastellen
## Mit mehreren Unterüberschriften deren Ausrichtung und einer die Einheit ersetzenden Größe
werte = arr([[1.25153244e+00, 1.38557443e+00, 1.40897994e+00],
             [2.39146223e-02, 1.56325105e-02, 2.27141900e-02],
             [1.40000000e+00, 1.40000000e+00, 1.64800000e+00],
             [1.00000000e-03, 1.00000000e-03, 5.00000000e-04]])
vergleichstabelle(werte, titel = 'Vergleich der Viskositäten nach Stokes bzw. Hagen-Poiseuille',
                  größen = ['Adiabatenkoeffizient', 'Clément-Desormes Luft', 'Rüchhardt Luft', 'Rüchhardt Argon'])
vergleichstabelle(werte, titel = 'Vergleich der Viskositäten nach Stokes bzw. Hagen-Poiseuille',
                  größen = ['Adiabatenkoeffizient', 'Clément-Desormes Luft', 'Rüchhardt        Luft', 'Rüchhardt        Argon'], 
                  ausrichtung = 'liste')


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 1: Vergleich der Viskositäten nach Stokes bzw. Hagen-Poiseuille

Adiabatenkoeffizient      Clément-Desormes Luft    Rüchhardt Luft    Rüchhardt Argon
----------------------  -----------------------  ----------------  -----------------
exp. Wert                                  1.25              1.39               1.41
dessen Fehler                              0.02              0.02               0.02
the. Wert                                  1.40              1.40               1.65
dessen Fehler                              0.00              0.00               0.00
rel. Abweichung [%]                      -10.60             -1.03             -14.50
Sigma                                      6.20              0.92              10.52
''';

Vergleich der Viskositäten nach Stokes bzw. Hagen-Poiseuille

Adiabatenkoeffizient       Clément-Desormes Luft    Rüchhardt Luft    Rüchhardt Argon 
------------------------   ---------------------   ----------------   ----------------
Wert       experimentell      1.252  ± 0.024        1.386  ± 0.016     1.409  ± 0.023 
           theoretisch        1.4000 ± 0.0010       1.4000 ± 0.0010    1.6480 ± 0.0005
Abweichung absolut           -0.148  ± 0.024       -0.014  ± 0.016    -0.239  ± 0.023 
           relativ                 -11 %                -1.0 %              -15 %     
           sigma                   6.2 σ                 0.92 σ              11 σ     

Vergleich der Viskositäten nach Stokes bzw. Hagen-Poiseuille

                       |              Wert               |             Abweichung            
 Adiabatenkoeffizient  | experimentell |   theoretisch   |    absolut     | relativ |  sigma 
------------------------------------------------------------------------------

#### Ausprobieren verschiedenster Einheit- und Größen-Kombinationen 

In [25]:
vergleichstabelle(werte, titel = 'Vergleich der Viskositäten nach Stokes bzw. Hagen-Poiseuille',
                  größen = ['Adiabatenkoeffizient', 'Clément-Desormes Luft', 'Rüchhardt Luft', 'Rüchhardt Argon'],
                  einheit = 'o.E.')

Eingabefehler: len(größen) = 4, aber es finden nur 3 Vergleiche statt.
Es darf höchstens so viele Größen wie Vergleiche geben.
(Ausnahme: einheit = '' und len(größen) = Anzahl Vergleiche + 1,
 dann wird der erste Wert als einheit interpretiert)



In [26]:
vergleichstabelle(werte, größen = ['Clément-Desormes Luft', 'Rüchhardt Luft', 'Rüchhardt Argon'])
vergleichstabelle(werte, größen = ['Clément-Desormes Luft', 'Rüchhardt        Luft', 'Rüchhardt        Argon'],
                  ausrichtung = 'liste')
vergleichstabelle(werte, größen = ['Clément-Desormes Luft', 'Rüchhardt        Luft', 'Rüchhardt        Argon'],
                  ausrichtung = 'liste', beschreibung = 'knapp')

                           Clément-Desormes Luft    Rüchhardt Luft    Rüchhardt Argon 
------------------------   ---------------------   ----------------   ----------------
Wert       experimentell      1.252  ± 0.024        1.386  ± 0.016     1.409  ± 0.023 
           theoretisch        1.4000 ± 0.0010       1.4000 ± 0.0010    1.6480 ± 0.0005
Abweichung absolut           -0.148  ± 0.024       -0.014  ± 0.016    -0.239  ± 0.023 
           relativ                 -11 %                -1.0 %              -15 %     
           sigma                   6.2 σ                 0.92 σ              11 σ     

                       |              Wert               |             Abweichung            
                       | experimentell |   theoretisch   |    absolut     | relativ |  sigma 
---------------------------------------------------------------------------------------------
Clément-Desormes Luft  | 1.252 ± 0.024 | 1.4000 ± 0.0010 | -0.148 ± 0.024 | -11   % |  6.2  σ
Rüchhardt     

In [27]:
vergleichstabelle(werte, größen = ['Rüchhardt Luft', 'Rüchhardt Argon'], einheit = 'o.E.')
vergleichstabelle(werte, größen = ['Rüchhardt        Luft', 'Rüchhardt        Argon'],
                  einheit = 'o.E.', ausrichtung = 'liste')

Einheit    o.E.             Rüchhardt Luft    Rüchhardt Argon                    
------------------------   ----------------   ----------------   ----------------
Wert       experimentell    1.252  ± 0.024     1.386  ± 0.016     1.409  ± 0.023 
           theoretisch      1.4000 ± 0.0010    1.4000 ± 0.0010    1.6480 ± 0.0005
Abweichung absolut         -0.148  ± 0.024    -0.014  ± 0.016    -0.239  ± 0.023 
           relativ               -11 %             -1.0 %              -15 %     
           sigma                 6.2 σ              0.92 σ              11 σ     

       Einheit         |              Wert               |             Abweichung            
         o.E.          | experimentell |   theoretisch   |    absolut     | relativ |  sigma 
---------------------------------------------------------------------------------------------
Rüchhardt        Luft  | 1.252 ± 0.024 | 1.4000 ± 0.0010 | -0.148 ± 0.024 | -11   % |  6.2  σ
Rüchhardt        Argon | 1.386 ± 0.016 | 1.4000 ±

In [28]:
vergleichstabelle(werte, größen = ['Rüchhardt Luft', '', 'Rüchhardt Argon'])
vergleichstabelle(werte, größen = ['Rüchhardt        Luft', '', 'Rüchhardt        Argon'],
                  ausrichtung = 'liste')

                            Rüchhardt Luft                       Rüchhardt Argon 
------------------------   ----------------   ----------------   ----------------
Wert       experimentell    1.252  ± 0.024     1.386  ± 0.016     1.409  ± 0.023 
           theoretisch      1.4000 ± 0.0010    1.4000 ± 0.0010    1.6480 ± 0.0005
Abweichung absolut         -0.148  ± 0.024    -0.014  ± 0.016    -0.239  ± 0.023 
           relativ               -11 %             -1.0 %              -15 %     
           sigma                 6.2 σ              0.92 σ              11 σ     

                       |              Wert               |             Abweichung            
                       | experimentell |   theoretisch   |    absolut     | relativ |  sigma 
---------------------------------------------------------------------------------------------
Rüchhardt        Luft  | 1.252 ± 0.024 | 1.4000 ± 0.0010 | -0.148 ± 0.024 | -11   % |  6.2  σ
                       | 1.386 ± 0.016 | 1.4000 ±

In [29]:
vergleichstabelle(werte, größen = 'Rüchhardt Luft')
vergleichstabelle(werte, größen = 'Rüchhardt        Luft',
                  ausrichtung = 'liste')

                            Rüchhardt Luft                                       
------------------------   ----------------   ----------------   ----------------
Wert       experimentell    1.252  ± 0.024     1.386  ± 0.016     1.409  ± 0.023 
           theoretisch      1.4000 ± 0.0010    1.4000 ± 0.0010    1.6480 ± 0.0005
Abweichung absolut         -0.148  ± 0.024    -0.014  ± 0.016    -0.239  ± 0.023 
           relativ               -11 %             -1.0 %              -15 %     
           sigma                 6.2 σ              0.92 σ              11 σ     

                      |              Wert               |             Abweichung            
                      | experimentell |   theoretisch   |    absolut     | relativ |  sigma 
--------------------------------------------------------------------------------------------
Rüchhardt        Luft | 1.252 ± 0.024 | 1.4000 ± 0.0010 | -0.148 ± 0.024 | -11   % |  6.2  σ
                      | 1.386 ± 0.016 | 1.4000 ± 0.00

In [30]:
vergleichstabelle(werte, einheit = 'o.E.')
vergleichstabelle(werte, einheit = 'o.E.', ausrichtung = 'liste')
vergleichstabelle(werte, einheit = 'o.E.', ausrichtung = 'liste', beschreibung = 'knapp')

Einheit    o.E.                                                                  
------------------------   ----------------   ----------------   ----------------
Wert       experimentell    1.252  ± 0.024     1.386  ± 0.016     1.409  ± 0.023 
           theoretisch      1.4000 ± 0.0010    1.4000 ± 0.0010    1.6480 ± 0.0005
Abweichung absolut         -0.148  ± 0.024    -0.014  ± 0.016    -0.239  ± 0.023 
           relativ               -11 %             -1.0 %              -15 %     
           sigma                 6.2 σ              0.92 σ              11 σ     

Einheit |              Wert               |             Abweichung            
 o.E.   | experimentell |   theoretisch   |    absolut     | relativ |  sigma 
------------------------------------------------------------------------------
        | 1.252 ± 0.024 | 1.4000 ± 0.0010 | -0.148 ± 0.024 | -11   % |  6.2  σ
        | 1.386 ± 0.016 | 1.4000 ± 0.0010 | -0.014 ± 0.016 |  -1.0 % |  0.92 σ
        | 1.409 ± 0.023 | 1.64

In [31]:
# Test: verschieden lange Einheiten
## erneuter Test für manuelle Rundung wegen theo-Fehler = 0
## Einsatz vom Faktor-Argument
werte = arr([1.39515138e-23, 2.90095704e-25, 1.38064900e-23, 0.00000000e+00])
vergleichstabelle(werte, titel = 'Literaturvergleich Boltzmannkonstante', einheit = '10^-23 J/K', faktor = 1e23)
vergleichstabelle(werte, titel = 'Literaturvergleich Boltzmannkonstante', einheit = '10^(-23) J / K', faktor = 1e23)
vergleichstabelle(werte, titel = 'Literaturvergleich Boltzmannkonstante', einheit = '10^(-23) J / K', 
                  beschreibung = 'knapp', faktor = 1e23)
vergleichstabelle(werte, einheit = 'µN', beschreibung = 'knapp', faktor = 1e23)
vergleichstabelle(werte, einheit = 'µN', faktor = 1e23)


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 1: Literaturvergleich Boltzmannkonstante

Angaben in 10^(-23) J/K
-------------------------  ----
Messwert                   1.40
dessen Fehler              0.03
Literaturwert              1.38
dessen Fehler              0.00
rel. Abweichung [%]        1.05
Sigma                      0.50
''';

Literaturvergleich Boltzmannkonstante

Einheit    10^-23 J/K                   
------------------------   -------------
Wert       experimentell   1.395 ± 0.029
           theoretisch     1.4   ± 0.0  
Abweichung absolut         0.015 ± 0.029
           relativ             1.1 %    
           sigma               0.50 σ   

Literaturvergleich Boltzmannkonstante

Einheit    10^(-23) J / K                
-------------------------   -------------
Wert       experimentell    1.395 ± 0.029
           theoretisch      1.4   ± 0.0  
Abweichung absolut          0.015 ± 0.029
           relativ              1.1 %    
           sigma                0.50 σ   

Literaturvergleich Boltzmannkonstante

[10^(-23) J / K]                
----------------   -------------
        Wert exp   1.395 ± 0.029
             the   1.4   ± 0.0  
        Abw. abs   0.015 ± 0.029
             rel       1.1 %    
             sig       0.50 σ   

    [µN]                
--------   -------------
Wert exp   1.395 ±

In [32]:
# Test: Listenansicht und theoretischer Wert ist 0
# Test: exakte Theo-Werte
werte = arr([[-0.0631769, 1.41696751, 2.48194946, 3.4566787, 4.55776173],
             [0.05460126, 0.05472070, 0.05496739, 0.0553094, 0.05582687],
             [0,          1.43030,    2.45902,    3.47089,   4.47741   ],
             [0,          0,          0,          0,         0         ]])
vergleichstabelle(werte, titel = 'Vergleich der Ordnungszahlen der Maxima', ausrichtung = 'liste')


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Vergleich der Ordnungszahlen der Maxima

  theoretisch    experimentell    Abweichung [%]    Sigma
-------------  ---------------  ----------------  -------
         0.00   -0.06 +/- 0.05            -inf        1.2
         1.43    1.42 +/- 0.05              -0.9      0.2
         2.46    2.48 +/- 0.05               0.9      0.4
         3.47    3.46 +/- 0.06              -0.4      0.3
         4.48    4.56 +/- 0.06               1.8      1.4
         
<ipython-input-7-ebcfbd735655>:12: RuntimeWarning: divide by zero encountered in true_divide
  abweichung_rel = abweichung / liste1 * 100   # [%], relative Abweichung
''';

Vergleich der Ordnungszahlen der Maxima

           Wert             |           Abweichung           
experimentell | theoretisch |   absolut    | relativ | sigma 
-------------------------------------------------------------
 -0.06 ± 0.05 |   0.0 ± 0.0 | -0.06 ± 0.05 |    -    | 1.2  σ
  1.42 ± 0.05 |   1.4 ± 0.0 | -0.01 ± 0.05 | -0.93 % | 0.24 σ
  2.48 ± 0.05 |   2.5 ± 0.0 |  0.02 ± 0.05 |  0.93 % | 0.42 σ
  3.46 ± 0.06 |   3.5 ± 0.0 | -0.01 ± 0.06 | -0.41 % | 0.26 σ
  4.56 ± 0.06 |   4.5 ± 0.0 |  0.08 ± 0.06 |  1.8  % | 1.4  σ



In [33]:
# Test: fehler_rel == fehler_sig == 0
werte = arr([[12368.47501413,  7421.08500848],
             [  623.31254737,   620.18815701],
             [12368.47501413,  7421.08500848],
             [ 1124.64056495,   836.7201341 ]])
vergleichstabelle(werte, ausrichtung = 'liste')


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Vergleich der Wellenzahl 2

  [mm^-1]       theoretisch     experimentell    Abweichung [%]    Sigma
---------  ----------------  ----------------  ----------------  -------
       a)  12.368 +/- 1.125  12.368 +/- 0.623              -0.0      0.0
       b)   7.421 +/- 0.837   7.421 +/- 0.620               0.0      0.0
''';

            Wert             |         Abweichung         
experimentell | theoretisch  | absolut  | relativ | sigma 
----------------------------------------------------------
  12400 ± 600 | 12400 ± 1100 | 0 ± 1300 |  0.00 % | 0.00 σ
   7400 ± 600 |  7400 ±  800 | 0 ± 1000 |  0.00 % | 0.00 σ



In [34]:
# Test: Einzelner Vergleich in Listenansicht
## beide Beschreibungvarianten in Listenansicht

werte = arr([11.7, 0.4, 6, 0.4])
vergleichstabelle(werte, ausrichtung = 'liste')
vergleichstabelle(werte, ausrichtung = 'liste', beschreibung = 'knapp')

           Wert             |         Abweichung         
experimentell | theoretisch |  absolut  | relativ | sigma
---------------------------------------------------------
   11.7 ± 0.4 |   6.0 ± 0.4 | 5.7 ± 0.6 |    95 % |  10 σ

         Wert          |          Abw.          
   exp     |    the    |    abs    | rel  | sig 
------------------------------------------------
11.7 ± 0.4 | 6.0 ± 0.4 | 5.7 ± 0.6 | 95 % | 10 σ



In [35]:
# Test: Faktorargument; experimenteller und Theoretischer wert sollen andere Namen haben
werte = arr([3580.,  100., 3080.,   90.])
vergleichstabelle(werte, faktor = 1e-3)


'''
Stand 2020-10-11: Umbenenung von theoretischem und experimentellem Wert noch nicht möglich
Es soll die Möglichkeit bestehen, Wert experimentell und Wert theoretisch selbst zu benennen.
'''

# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Tabelle 2: Vergleich der Grenzfrequenzen aus der Hochpass- und Tiefpassbestimmung

Grenzfrequenz      [kHz]
---------------  -------
Hochpass            3.58
dessen Fehler       0.10
Tiefpass            3.08
dessen Fehler       0.09
Abweichung          0.50
Sigma               3.72
''';

------------------------   -----------
Wert       experimentell   3.58 ± 0.10
           theoretisch     3.08 ± 0.09
Abweichung absolut         0.50 ± 0.13
           relativ             16 %   
           sigma              3.7 σ   



### Tests mit fehlenden Werten

In [36]:
# Test: Vergleiche, in denen Werte fehlen
werte = arr([[820.5, 571.4, 501.0, 470.2, 454.3, None, 433.6, 430.7, None, 423.5],
             [0.8493218002880191, 1.1890505204032267, 0.5945252602016133,
              1.2739827004320285, 1.6986436005760381, None, 2.1233045007200477,
              2.1233045007200477, None, 1.2739827004320285],
             [820.5, 570.7094072095474, 500.2226094489081, 468.7724747247101,
              451.650407609065, 441.1913744753898, 434.29622148706073, 429.4949134280389,
              426.0102653247866, 423.39753086740905],
             [0.8493218002880192, 0.4109085361760692, 0.3156761241481166,
              0.2772294138792619, 0.2573474764388973, 0.24556650385495118,
              0.23795081828704262, 0.23271862938516857, 0.2289576870796812,
              0.22615788961099104]])
vergleichstabelle(werte, ausrichtung = 'liste')


# Zum Vergleich: Resultat mit der ursprünglischen Funktion
'''
Linien der 1. Nebenserie [nm]

erwartet       gemessen         Abweichung    Sigma
-------------  -------------  ------------  -------
820.5 +/- 0.8  820.5 +/- 0.8           0.0      0.0
570.7 +/- 0.4  571.4 +/- 1.2           0.7      0.5
500.2 +/- 0.3  501.0 +/- 0.6           0.8      1.2
468.8 +/- 0.3  470.2 +/- 1.3           1.4      1.1
451.7 +/- 0.3  454.3 +/- 1.7           2.6      1.5
441.2 +/- 0.2
434.3 +/- 0.2  433.6 +/- 2.1          -0.7      0.3
429.5 +/- 0.2  430.7 +/- 2.1           1.2      0.6
426.0 +/- 0.2
423.4 +/- 0.2  423.5 +/- 1.3           0.1      0.1
''';

            Wert              |           Abweichung           
experimentell |  theoretisch  |  absolut   | relativ  |  sigma 
---------------------------------------------------------------
  820.5 ± 0.8 | 820.5  ± 0.8  |  0.0 ± 1.2 |  0.00  % | 0.00  σ
  571.4 ± 1.2 | 570.7  ± 0.4  |  0.7 ± 1.3 |  0.12  % | 0.55  σ
  501.0 ± 0.6 | 500.22 ± 0.32 |  0.8 ± 0.7 |  0.16  % | 1.2   σ
  470.2 ± 1.3 | 468.77 ± 0.28 |  1.4 ± 1.3 |  0.30  % | 1.1   σ
  454.3 ± 1.7 | 451.65 ± 0.26 |  2.6 ± 1.7 |  0.59  % | 1.5   σ
              | 441.19 ± 0.25 |            |          |        
  433.6 ± 2.1 | 434.30 ± 0.24 | -0.7 ± 2.1 | -0.16  % | 0.33  σ
  430.7 ± 2.1 | 429.49 ± 0.23 |  1.2 ± 2.1 |  0.28  % | 0.56  σ
              | 426.01 ± 0.23 |            |          |        
  423.5 ± 1.3 | 423.40 ± 0.23 |  0.1 ± 1.3 |  0.024 % | 0.079 σ



In [37]:
vergleichstabelle((werte.T[-5:]).T)

------------------------   -------------   -------------   -------------   -------------   -------------
Wert       experimentell                   433.6  ± 2.1    430.7  ± 2.1                    423.5  ± 1.3 
           theoretisch     441.19 ± 0.25   434.30 ± 0.24   429.49 ± 0.23   426.01 ± 0.23   423.40 ± 0.23
Abweichung absolut                          -0.7  ± 2.1      1.2  ± 2.1                      0.1  ± 1.3 
           relativ                            -0.16 %          0.28 %                         0.024 %   
           sigma                               0.33 σ          0.56 σ                         0.079 σ   



In [38]:
werte = arr([[0.0372, 0.348, 1.579, 4.382], [0.000571, 0.368, None, None], [0.0466, 0.295, 1.598, 4.234], [0.000395, None, 0.456, None]])
vergleichstabelle(werte)
vergleichstabelle(werte, ausrichtung = 'liste')

------------------------   ------------------   -----------   ----------   ---------
Wert       experimentell    0.0372  ± 0.0006    0.35 ± 0.37    1.6 ± 0.0   4.4 ± 0.0
           theoretisch      0.04660 ± 0.00040   0.3  ± 0.0     1.6 ± 0.5   4.2 ± 0.0
Abweichung absolut         -0.0094  ± 0.0007    0.05 ± 0.37   -0.0 ± 0.5   0.1 ± 0.0
           relativ                -20 %             18 %        -1.2 %       3.5 %  
           sigma                   14 σ            0.14 σ       0.042 σ        -    

               Wert                 |              Abweichung              
 experimentell  |    theoretisch    |     absolut      | relativ |  sigma  
---------------------------------------------------------------------------
0.0372 ± 0.0006 | 0.04660 ± 0.00040 | -0.0094 ± 0.0007 | -20   % | 14     σ
0.35   ± 0.37   | 0.3     ± 0.0     |  0.05   ± 0.37   |  18   % |  0.14  σ
1.6    ± 0.0    | 1.6     ± 0.5     | -0.0    ± 0.5    |  -1.2 % |  0.042 σ
4.4    ± 0.0    | 4.2     ± 0.0  

In [39]:
werte = arr([[None, None], [None, 4], [58, 39], [2.3, None]])
vergleichstabelle(werte, ausrichtung = 'liste', beschreibung = 'knapp')
print('')

werte = arr([[58, 39], [2.3, 5.2], [None, None], [3, 4]])
vergleichstabelle(werte, ausrichtung = 'liste', beschreibung = 'knapp')
vergleichstabelle(werte, ausrichtung = 'blöcke', beschreibung = 'knapp')
print('')

werte = arr([[None, None], [2.3, 5.2], [None, None], [3, 4]])
vergleichstabelle(werte, ausrichtung = 'liste', beschreibung = 'knapp')
vergleichstabelle(werte, ausrichtung = 'blöcke', beschreibung = 'knapp', größen = 'Test')

      Wert       |      Abw.      
exp |    the     | abs | rel | sig
----------------------------------
    | 58.0 ± 2.3 |     |     |    
    | 39.0 ± 0.0 |     |     |    


      Wert       |      Abw.      
   exp     | the | abs | rel | sig
----------------------------------
58.0 ± 2.3 |     |     |     |    
39   ± 5   |     |     |     |    

--------   ----------   ------
Wert exp   58.0 ± 2.3   39 ± 5
     the                      
Abw. abs                      
     rel                      
     sig                      


  Wert    |      Abw.      
exp | the | abs | rel | sig
---------------------------
    |     |     |     |    
    |     |     |     |    

           Test   
--------   ----   
Wert exp          
     the          
Abw. abs          
     rel          
     sig          



## Skizzen zu Darstellungsvarianten